## 自动求导

    PyTorch中，所有神经网络的核心是autograd包。autograd包为张量上的所有操作提供了自
    动求导机制。它是一个在运行时定义（define-by-run）的框架，这意味着反向传播是根据代
    码如何运行来决定的，并且每次迭代可以是不同的。

    本节学习：Autograd的求导机制
            梯度的反向传播

    torch.Tensor是这个包的核心类。如果设置了它的属性 .requires_grad为True，那么它
    将会追踪对于该张量的所有操作。当计算完成后可以通过调用 .backward()，来自动计算所
    有的梯度。这个张量的所有梯度会自动累加到 .grad属性。

    注意：在y.backward（）时，如果y是标量，则不需要为backward（）传入任何参数；否则
         需要传入一个与y同形的Tensor。

    要阻止一个张量被跟踪历史，可以调用 .detach()方法将其与计算历史分离，并阻止它未来的
    计算记录被跟踪。为了防止跟踪历史记录（和使用内存），可以将代码块包装在with 
    torch.no_grad（）：中。在评估模型时特别有用，因为模型可能具有requires_grad = 
    True 的可训练的参数，但是我们不需要在此过程中对他们进行梯度计算。

    还有一个类对于autograd的实现非常重要：Function。Tensor和Function互相连接生成
    了一个无环图（acylic graph），它编码了完整的计算历史。每个张量都有一个 .grad_fn
    属性，该属性引用了创建Tensor自身的Funtion（除非这个张量使用户手动创建的，即这个张
    量的grad_fn是None）。下面给出的例子中，张量由用户手动创建，因此grad_fn返回结果为None

In [1]:
import torch
x = torch.randn(3, 3, requires_grad=True)
print(x.grad_fn)

None


    如果需要计算导数，可以在Tensor上调用 .backward（）。如果Tensor是一个标量（即它
    包含一个元素的数据），则不需要为backward（）指定任何参数，但是如果它有更多的元素，
    则需要指定一个gradient参数，该参数是形状匹配的张量。

In [4]:
# 创建一个张量，并设置requires-grad = True用来追踪其计算历史
x = torch.ones(2, 2, requires_grad=True)
print(x)
# 对这个张量做一次运算
y = x ** 2
print(y)
# y是计算的结果，所以它有grad_fn属性
print(y.grad_fn)
# 对y进行更多的操作
z = y * y * 3
out = z.mean()
print(z)
print(out)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[1., 1.],
        [1., 1.]], grad_fn=<PowBackward0>)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<MulBackward0>)
tensor(3., grad_fn=<MeanBackward0>)


    .requires_grad_(...) 原地改变了现有张量的requires_grad标志。如果没有指定的话
    ，默认输入的这个标志是 False。